In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

c:\Users\Carlos Weffer\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Carlos Weffer\anaconda3\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
c:\Users\Carlos Weffer\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
file_path = "label_data/label_text_Adrian.csv"
df = pd.read_csv(file_path, header = None)

In [3]:
import csv

from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

In [4]:
import spacy

# Load pretrained models
weight_pipe= pipeline("ner", model="d4data/biomedical-ner-all", aggregation_strategy="average")
food_pipe = pipeline("ner", model="Dizex/InstaFoodRoBERTa-NER", aggregation_strategy="max")
date_pipe = pipeline("ner", model="mdarhri00/named-entity-recognition", aggregation_strategy="average")


In [8]:
def text_extraction(text):

    # Apply the pretrained models
    lab_value_ents = weight_pipe(text, aggregation_strategy="average")
    date_ents = date_pipe(text)
    food_ents = food_pipe(text)


    food_name = "No Food Found"
    packaged_date = "No Package Date Found"
    expiry_date = "No Expiry Date Found"
    weight = "No Weight Found"


    # Getting Dates
    #print("DATE ENT LIST:")
    for ent in date_ents:
        if ent['entity_group'] == "date_time":
            #print(f"ent: {ent['word']}, class: {ent['entity_group']}")
            start = ent["start"]
            end = ent["end"]
            
            # Get the tokens before and after the DATE entity
            before_ent = text[start-20:start]
            after_ent = text[end:end+5]
            
            # Check the context around the DATE entity
            context = before_ent.lower() + " " + after_ent.lower()

            # Define the phrases to look for
            packaging_phrases = ["packed", "pkg on", "packaged on", 'production date']
            expiry_phrases = ["best before", "use by", "expiry date", "expires on", "expires by", 'expires', 'after']

            if any(phrase in context for phrase in expiry_phrases):
                expiry_date = ent["word"]
            elif any(phrase in context for phrase in packaging_phrases):
                packaged_date = ent["word"]


    # Getting Weight

    # print("")
    #print("WEIGHT ENT LIST:")
    #[print(f"ent: {ent['word']}, score: {ent['score']}, class: {ent['entity_group']}") for ent in lab_value_ents]
    weight_phrases = ['0g', '1g','2g','3g','4g','5g','6g','7g','8g', '9g', 'kg','grams','kilograms']
    banned_weight_phrases = ["-", "kg ", "g "]

    weight_entities = [ent for ent in lab_value_ents if any(phrase in ent['word'].lower() for phrase in weight_phrases) and not any(phrase in ent['word'].lower() for phrase in banned_weight_phrases)]
    #[print(f"ent: {ent['word']}, score: {ent['score']}") for ent in weight_entities]
    max_score_entity = max(weight_entities, key=lambda entity: entity['score'])
    weight = max_score_entity["word"]


    # Getting Food Name

    # print("")
    # print("FOOD ENT LIST:")
    #[print(f"ent: {ent['word']}, score: {ent['score']}") for ent in food_ents]

    max_score_entity = max(food_ents, key=lambda entity: entity['score'])
    food_name = max_score_entity["word"]


    return expiry_date, packaged_date, weight, food_name

In [9]:
text = df[0][1]

expiry_date, packaged_date, weight, food_name = text_extraction(text)

print("")
print(f"Food Name: {food_name}")
print(f"Weight: {weight}")
print(f"Packaged Date: {packaged_date}")
print(f"Expiry Date: {expiry_date}")
print("")
print(f"Example Text: {text}")


Food Name:  Australia Beef Burger Bulk Pack
Weight: 13. 63 kg
Packaged Date: 14 May 2024
Expiry Date: 14 May 2025

Example Text: Keep Frozen Product of Australia Beef Burger Bulk Pack Net Wt 13.63 kg 13.63 KG Pkg On 14 May 2024 Best Before 14 May 2025 Top Cut Foods Pty Ltd 101265 Boneless Beef NL Allergens


In [7]:
# Code which reads the results into a CSV file, saving here for later use


# with open("label_data/label_text_Adrian.csv", mode='r', encoding='utf-8') as file:
#     csv_reader = csv.reader(file)
#     with open("extracted_info.csv", mode='w', newline='', encoding='utf-8') as output_file:
#         csv_writer = csv.writer(output_file)
#         csv_writer.writerow(["Food Name", "Weight", "Expiry Date"])
#         for row in csv_reader:
#             text = row[0]
#             food_name, packaged_date, expiry_date = extract_information(text)
#             csv_writer.writerow([food_name, packaged_date, expiry_date])

# print("Information extraction completed.")